In [ ]:
!pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]>/dev/null

In [ ]:
!pip install --upgrade accelerate >/dev/null

In [ ]:
import pandas as pd
import torch

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("wmt16", "ro-en")
metric = load_metric("sacrebleu")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/610320 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-4-b4856c86bc22>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'The World Bank stated that 17 of the G20 countries had, in total, introduced 47 new trade barriers and tariffs.', 'ro': 'Banca Mondială a declarat că 17 dintre statele G20 introduseseră, în total, 47 de noi obstacole şi taxe comerciale.'}"
1,"{'en': 'Firstly, we need a strong European policy and to strategically reassert Europe's commitment to being a global player in the food and agricultural sectors.', 'ro': 'Prima este că avem nevoie de o politică europeană puternică și că trebuie să reafirmăm în mod strategic angajamentul Europei de a fi un actor internațional în sectorul alimentar și agricol.'}"
2,"{'en': 'If Lisbon enters into force, Europe must draw the following lesson: let the Union act pragmatically and let it repeatedly prove its added value.', 'ro': 'Dacă Tratatul de la Lisabona intră în vigoare, Europa trebuie să înveţe următoarea lecţie: să lăsăm Uniunea să acţioneze pragmatic şi să o lăsăm să-şi dovedească valoarea adăugată.'}"
3,"{'en': 'The European Commission okayed the tender documentation for building a second bridge over the Danube, spanning Vidin, Bulgaria and Kalafat, Romania.', 'ro': 'Comisia Europeană a aprobat documentaţia licitaţiei pentru construcţia unui al doilea pod peste Dunăre între Vidin, Bulgaria, şi Calafat, România.'}"
4,"{'en': 'We are clearly seeing Islam becoming a state religion.', 'ro': 'Vedem în mod clar islamul devenind o religie de stat.'}"


In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

In [ ]:
model_checkpoint = "facebook/m2m100_418M"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "facebook/m2m100_418M", src_lang="en", tgt_lang="ar"
)

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[128022, 65761, 4, 15911, 13331, 8931, 6226, 30, 2], [128022, 36606, 117, 122000, 8931, 6226, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
from transformers import (
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    DefaultDataCollator,
    DataCollatorWithPadding,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import TrainingArguments, Trainer

source_lang = "en"
target_lang = "ar"
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
flores_eng_ar = pd.read_excel("/content/eng_Latn_arb_Arab.devtest.xlsx")
flores_eng_ar.shape

(1012, 3)

In [ ]:
class EnglishArabicDataset(torch.utils.data.Dataset):
    def __init__(self, df, max_len=16):
        self.max_length = max_len
        self.df = df
        self.df = self.df[~self.df["Arabic"].isna()].reset_index()
        self.tokenizer = tokenizer = AutoTokenizer.from_pretrained(
            "facebook/m2m100_418M", src_lang="en", tgt_lang="ar"
        )

    def __len__(self):
        return self.df.shape[0] - 1

    def __getitem__(self, idx):
        src = self.df.loc[idx, "English"]
        tgt = self.df.loc[idx, "Arabic"]
        model_inputs = self.tokenizer(
            src,
            text_target=tgt,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt",
        )
        return {
            "input_ids": model_inputs["input_ids"][0, :],
            "labels": model_inputs["labels"][0, :],
        }

In [ ]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ro"


def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenizer(, max_length=max_input_length, truncation=True)

In [ ]:
train_set_torch = EnglishArabicDataset(flores_eng_ar)

In [ ]:
train_set_torch[1011]

{'input_ids': tensor([128022,  59895,    397,  30660, 125553,   6776,  73201,  45268,   2002,
             12,  29304,   1490,    193,   7518, 121113,      2]),
 'labels': tensor([128006,   1324,    874,    727,  52694,    382,   2472,  14631,   1532,
          27038,   2892,   1093,  10871, 108446,    606,      2])}

In [ ]:
len(train_set_torch)

1012

In [ ]:
dataset = load_dataset("wmt16", "ro-en", split="train")

In [ ]:
dataset["translation"]

In [ ]:
from datasets import DatasetDict

dataset_dict = DatasetDict({"train": train_set})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
# from transformers import DataCollator
# data_collator = DataCollator.from_config(tokenizer)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'NewType' object has no attribute 'from_config'

In [ ]:
batch = data_collator(train_set_torch)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_set_torch,
    # eval_dataset=train_set_torch,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
train_set_torch[1023]

In [ ]:
trainer.train()

In [ ]:
from datasets import load_dataset
from transformers import DataCollatorWithPadding, AutoTokenizer

# Load the dataset
dataset = load_dataset("imdb", split="train")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Create an instance of DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Collate the dataset into batches
batches = data_collator(dataset)

# Print the first batch
print(batches[0])

In [ ]:
from transformers import (
    BertTokenizerFast,
    DataCollatorWithPadding,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from torch.utils.data import Dataset
import torch

# 1. Tokenization
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
samples = ["Hello, this is an example", "This is another longer example sentence"]
labels = [0, 1]  # sample binary labels

tokenized_inputs = tokenizer(
    samples, truncation=True, padding=False, return_tensors="pt"
)

# 2. DataCollator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


# 3. Dataset
class MyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings.input_ids)


dataset = MyDataset(tokenized_inputs, labels)

# 4. Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# 5. Training Arguments
training_args = TrainingArguments(
    output_dir="./results",  # output directory
    num_train_epochs=3,  # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir="./logs",  # directory for storing logs
)

# 6. Trainer
trainer = Trainer(
    model=model,  # the instantiated Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=dataset,  # training dataset
    eval_dataset=dataset,  # evaluation dataset
    data_collator=data_collator,  # use our custom data collator
)

# 7. Train
trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss


TrainOutput(global_step=3, training_loss=0.7013418674468994, metrics={'train_runtime': 5.6076, 'train_samples_per_second': 1.07, 'train_steps_per_second': 0.535, 'total_flos': 24666661440.0, 'train_loss': 0.7013418674468994, 'epoch': 3.0})

In [ ]:
from transformers import (
    BertTokenizerFast,
    DataCollatorWithPadding,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from torch.utils.data import Dataset
import torch

model_checkpoint = "facebook/m2m100_418M"
# 1. Tokenization
tokenizer = AutoTokenizer.from_pretrained(
    "facebook/m2m100_418M", src_lang="en", tgt_lang="ar"
)
samples = ["Hello, this is an example", "This is another longer example sentence"]
labels = [0, 1]  # sample binary labels

# tokenized_inputs = tokenizer(samples, truncation=True, padding=False, return_tensors="pt")

# 2. DataCollator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# 3. Dataset
class EnglishArabicDataset(torch.utils.data.Dataset):
    def __init__(self, df, max_len=16):
        self.max_length = max_len
        self.df = df
        self.df = self.df[~self.df["Arabic"].isna()].reset_index()
        self.tokenizer = AutoTokenizer.from_pretrained(
            "facebook/m2m100_418M", src_lang="en", tgt_lang="ar"
        )

    def __len__(self):
        return self.df.shape[0] - 1

    def __getitem__(self, idx):
        src = self.df.loc[idx, "English"]
        tgt = self.df.loc[idx, "Arabic"]
        model_inputs = self.tokenizer(
            src,
            text_target=tgt,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt",
        )
        return {
            "input_ids": model_inputs["input_ids"][0, :],
            "labels": model_inputs["labels"][0, :],
        }


dataset = EnglishArabicDataset(flores_eng_ar)
print(dataset[100])

{'input_ids': tensor([128022,    593,  13331,  63245,     12,     55,  12293,      4,     48,
          4909, 116982,   3419,   1961,   4909,   7798,      2]), 'labels': tensor([128006,  11000,   8424,  22295,     52,   7922,  59258,  20281,  53824,
          1206,    874,  63348,   1625,   6969,    301,      2])}


In [ ]:
# 3. Dataset
class EnglishSudaneseDataset(torch.utils.data.Dataset):
    def __init__(self, df, max_len=16):
        self.max_length = max_len
        self.df = df
        self.df = self.df[~self.df["Sudanese"].isna()].reset_index()
        self.tokenizer = AutoTokenizer.from_pretrained(
            "facebook/m2m100_418M", src_lang="en", tgt_lang="ar"
        )

    def __len__(self):
        return self.df.shape[0] - 1

    def __getitem__(self, idx):
        src = self.df.loc[idx, "English"]
        tgt = self.df.loc[idx, "Sudanese"]
        model_inputs = self.tokenizer(
            src,
            text_target=tgt,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt",
        )
        return {
            "input_ids": model_inputs["input_ids"][0, :],
            "labels": model_inputs["labels"][0, :],
        }


flores_eng_sd = pd.read_excel(
    "/content/eng_Latn_devtest_translated_0_400_401_500_601_699_701_800_801_1011_final.xlsx"
)
madar_eng_sd = pd.read_excel("/content/madar_translated.xlsx")
dataset = EnglishSudaneseDataset(madar_eng_sd)
print(dataset[100])

{'input_ids': tensor([128022,    203,  13635,   1900,   1253, 118504,      5,      2,      1,
             1,      1,      1,      1,      1,      1,      1]), 'labels': tensor([128006,   2122,  13780,  18962,   2375,  27508,    680,  66089,      5,
             2,      1,      1,      1,      1,      1,      1])}


In [ ]:
[idx for idx in range(len(dataset)) if dataset[idx]["input_ids"].shape[0] != 16]

[]

In [ ]:
# 4. Model
device = "cuda" if torch.cuda.is_available else "cpu"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model = model.to(device)
# 5. Training Arguments
training_args = TrainingArguments(
    output_dir="./results",  # output directory
    num_train_epochs=10,  # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir="./logs",  # directory for storing logs
)

# 6. Trainer
trainer = Trainer(
    model=model,  # the instantiated Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=dataset,  # training dataset
    eval_dataset=dataset,  # evaluation dataset
    data_collator=data_collator,  # use our custom data collator
    compute_metrics=compute_metrics,
)

# 7. Train
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.485400
1000,0.403400


TrainOutput(global_step=1250, training_loss=1.1722997619628905, metrics={'train_runtime': 868.2411, 'train_samples_per_second': 23.024, 'train_steps_per_second': 1.44, 'total_flos': 676881578065920.0, 'train_loss': 1.1722997619628905, 'epoch': 10.0})

In [ ]:
def translate(sentence, tokenizer, model):
    model.eval()
    tokenized_sentence = tokenizer(sentence, return_tensors="pt").to(device)
    generated_tokens = model.generate(
        **tokenized_sentence, forced_bos_token_id=tokenizer.get_lang_id("ar")
    )
    translated_sentence = tokenizer.batch_decode(
        generated_tokens, skip_special_tokens=True
    )
    model.train()
    return translated_sentence[0]

In [ ]:
translate(
    "We now have 4-month-old mice that are non-diabetic that used to be diabetic, he added.",
    tokenizer,
    model,
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'الآن عندنا خمرة عمرها أربعة أشهر غير مرض'

In [ ]:
pred = (
    flores_eng_sd[~flores_eng_sd["Sudanese"].isna()]
    .reset_index()
    .iloc[0:50, :]["English"]
    .apply(lambda x: translate(x, tokenizer, model))
    .tolist()
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
flores_eng_sd[~flores_eng_sd["Sudanese"].isna()].reset_index().iloc[0:50, :]["English"][
    0
]

'"We now have 4-month-old mice that are non-diabetic that used to be diabetic," he added.\n'

In [ ]:
gt = flores_eng_sd.iloc[0:50, :]["Sudanese"].tolist()

In [ ]:
pred[0:100]

['"الآن عندنا خمرة عمرها أربعة أشهر غير',
 'الدكتور ايهود اور، أستاذ طب في جامعة دالوسي',
 'تمام مع بعض الخبراء التانيين، هو متشك حول',
 'في ايام الاتنين، سارة دانيوس، الأمين الدائم ل',
 'دانيوس قال، "الآن نفعل شي. اتصلت',
 'سابق، المدير التنفيذي لشركة رينج، جي',
 'هو بنى مكالمة باب الواي فاي، قال.',
 'سيدينوف قال ان المبيعات زيادة بعد ظهوره في',
 'في اخر عام 2017، سيمينوف ظهرت في',
 'انجليس حلت محاكمة مع شركة أمنية منافسة',
 'في حين يبدو أن لقاح تجريبي واحد قادر يقلل',
 'كوكتيل واحد لمكافحة الجسد، زام',
 'في تجارب البارم، زامب عملت كمراقب',
 'اللياقة البدنية في الولايات. تدعم رسالة المجموع',
 'اوكي، سيدي. أنا موافق على بيان اوزوك ربما',
 'الولايات. الجمنازية تدعم تحقيقات مستقلة احتمال ت',
 'اللياقة البدنية في الولايات. و اللياقة البدنية في',
 'من خلال الستينات، عملت برزينسكي ل',
 'خلال الانتسابات لعام 1976، مشير كارتر في السياسة',
 'كشرطة الأمن، مساعد كارتر في الشؤون العالمية، زي',
 'الفلم، بتاع رايان غاسلين و ايما',
 'Gosling و Stone حصلوا على ترشيحات لأفضل مم',
 'ترشيحات تانية تشمل 

In [ ]:
gt[0:100]

['هسي عندنا فيران عمرها ٤ شهور ما مصابة بداء السكري و الاعتادت انها تكون مصابة بداء السكري.',
 'الدكتور اليهود اور  (استاذ الطب بجامعة دالهوزي في هاليفاكس ، نوفا سكوتيا و رئيس القسم الاكلينكي والعلمي في جمعية السكري الكندية من انو البحث لسه في أيامه الأولى.',
 'زيو و زي شوية من الخبراء التانيين،، زول شكاك بخصوص قصة علاج مرض السكري ، وبقول انو النتايج دي ما عندها علاقة بالناس الجد جد العندهم مرض السكري ( بياخدوا انسولين) .',
 'يوم الاتنين ،سارة دانيوس قالت ( السكرتيرة الدائمة للجنة نوبل للأدب في ألاكاديمية السويدية ، في الردايو  sveriges  فالسويد ، بانو اللجنة دي ما قادرة على أنها تصل ل بوب ديلان عشان يتفاهموا معاهو في قصة الفوز بجائزة نوبل في الأدب لعام ٢٠١٦ ، و استغنى عن جهوده في انو يصل ليهو.',
 'دانيوس قال "  حاليا نحن ما بنعمل في شي ، انا ضربت و رسلت رسايل ف الايميل لاقرب زول بساعده و جاتني ردود ودودة شديد ، حاليا ، و دا اكيد طبعا كفاية ".',
 'فالمرة الفاتت ، لاحظ جيمي سيمينوف ، الرئيس التنفيذي لشركة Ring  ، انو الشركة دي بدت لمن جرس الباب ما كان بتسمع في دكان.',
 'قال  قمت عملت جر

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

references = [
    [g.split()] for g in gt
]  # [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
candidates = [p.split() for p in pred]  # ['this', 'is', 'a', 'test']
print(reference[:10], candidate[:10])
score = sum(
    [
        sentence_bleu(reference, candidate, weights=[1])
        for reference, candidate in zip(references, candidates)
    ]
)
print(score / len(candidates))

[['وقد', 'تلقى', 'الفيلم', 'الذي', 'يضم', 'رايان', 'غوسلينغ', 'وإيما', 'ستون،', 'ترشيحات', 'في', 'جميع', 'الفئات', 'الرئيسية.']] ['حصل', 'الفيلم،', 'الذي', 'يضم', 'رايان', 'غوسلينج', 'وإيما', 'ستون،', 'على', 'ترشيحات']
0.05848118970765511
